To select code search queries for the evaluation, we build a benchmark of queries from Python programming questions in Stack Overflow. 

The StackOverflow questions should satisfy the following conditions:
1. Data Between **2017-01-01 to 2019-02-01**
2. The question is a Python programming task, including any versions of Python 
3. The question score to that question is **higher than or equal to 0**
4. The question does have a best answer
5. The best answer to that question has **exactly one def function** answer
6. The question is not a 'TypeError' or a 'why' question, but a 'how to' question  


In [1]:
import pandas as pd

In [20]:
def trns(value):
    s = value.split("<code>")[1].split("</code>")[0]
    return s

In [18]:
def get_question_answer_pair(df):
    idx = []
    for i in range(len(df)):
        if df['Best_Answer'][i].count('<code>def ') == 1 and df['Best_Answer'][i].count('<code>') == 1:
            idx.append(i)

    df_qa = pd.DataFrame({
        "Post_Link_ID": df.loc[idx]['Post_Link_Id'],
        "Question_Score": df.loc[idx]['Question_Score'],
        "Question_Title": df.loc[idx]['Question_Title'], 
        "Question_Content": df.loc[idx]['Question_Content'],
        "Answer": df.loc[idx]['Best_Answer'].apply(trns),
        "Tags": df.loc[idx]['Tags']
        })
    
    df_qa = df_qa[df_qa['Question_Score'] >= 0].sort_values(by = 'Question_Score', ascending=False)
    
    return df_qa

### Data Processing

In [13]:
df = pd.read_csv('./data/QueryResults.csv')
print(df.shape)

(50000, 8)


In [16]:
#df['date'] = df.Create_Date.apply(lambda x: str(x).split(' ')[0])
#df.date.unique

In [21]:
df = get_question_answer_pair(df)
print(df.shape)

(2857, 6)


In [22]:
print("Python Only Questions:", len(df[((df['Tags'] == '<python>'))|(df['Tags'] == '<python-3.x>')]))
print("Python with Package Questions:", len(df) - len(df[((df['Tags'] == '<python>'))|(df['Tags'] == '<python-3.x>')]))

Python Only Questions: 257
Python with Package Questions: 2600


In [ ]:
export_csv = df.to_csv (r'./data/StackOverFlow.csv', index = None, header=True)

### Reference

In [ ]:
# Example Question
pid = 50253517
str(df[df['Post_Link_ID'] == pid].Answer).split('\n')

In [ ]:
# Example Answer
df_with_filter.Answer[idx].split("\n")

In [ ]:
# SQL Code
SELECT Posts.Id as [Post_Link_Id], 
Posts.Score as [Question_Score],
Posts.ViewCount,
Posts.Title As [Question_Title],
Posts.Body As [Question_Content],
Answers.Body AS [Best_Answer],
Posts.Tags AS [Tags],
Posts.CreationDate AS [Create_Date]

From Posts
left join Posts As Answers 
        ON Answers.parentid = Posts.id
        AND Answers.posttypeid = 2

WHERE Posts.PostTypeId = 1 AND 
YEAR(Posts.CreationDate) BETWEEN 2017 AND 2019 AND
Posts.AcceptedAnswerId = Answers.Id AND
Posts.Tags LIKE '%python%' AND
Posts.Title NOT LIKE '%why%' AND 
Posts.Title NOT LIKE '%Why%' AND
Posts.Title NOT LIKE '%Error%' 

GROUP BY
Posts.Id, Posts.CreationDate, Posts.Score, Posts.ViewCount, Posts.Title, Posts.ViewCount, Answers.Body, Posts.Body, Posts.AcceptedAnswerId, Answers.Id, Posts.Tags

HAVING Answers.Body LIKE '%def %'


